<a href="https://colab.research.google.com/github/appliedcode/mthree-c422/blob/mthree-422-salleh/Exercises/day-13/AI-Ethics/Production-Ready-Practice-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Problem Set: Operationalizing Ethics in AI Development & Deployment — Ride‑Hailing Driver Evaluation

We’ll simulate an AI system that rates and flags ride‑hailing drivers for review based on their trip history and customer feedback. \
The goal: embed and operationalize ethics from design through to post‑deployment monitoring.


#1. Dataset Creation

In [2]:
import pandas as pd
import numpy as np

np.random.seed(77)

# Synthetic Dataset for Ride-Hailing Drivers
n = 1500
genders = ["Male", "Female", "Other"]
cities = ["Metro", "Urban", "Town"]

data = pd.DataFrame({
    "Driver_ID": range(1, n+1),
    "Age": np.random.randint(21, 65, n),
    "Gender": np.random.choice(genders, n, p=[0.6, 0.35, 0.05]),
    "City_Type": np.random.choice(cities, n, p=[0.4, 0.45, 0.15]),
    "Trips_Completed": np.random.randint(50, 2000, n),
    "Avg_Rating": np.round(np.random.uniform(3.0, 5.0, n), 2),  # rating out of 5
    "Complaints": np.random.randint(0, 20, n),
    "Accidents": np.random.randint(0, 5, n)
})

# Introduce bias: drivers from Metro more likely to be flagged for review
data["Flagged_For_Review"] = np.where(
    (data["Complaints"] > 5) | (data["Accidents"] > 0),
    np.random.choice([1, 0], n, p=[0.7, 0.3]),
    np.random.choice([1, 0], n, p=[0.15, 0.85])
)

mask_metro = data["City_Type"] == "Metro"
metro_draws = np.random.choice([1, 0], size=len(data), p=[0.4, 0.6])
data["Flagged_For_Review"] = np.where(
    mask_metro, metro_draws, data["Flagged_For_Review"]
)

data.head()

,Driver_ID,Age,Gender,City_Type,Trips_Completed,Avg_Rating,Complaints,Accidents,Flagged_For_Review
0,1,44,Other,Metro,324,3.88,12,3,1
1,2,52,Male,Urban,323,3.96,17,3,0
2,3,41,Female,Urban,448,3.53,4,4,1
3,4,41,Female,Town,1485,4.83,19,1,0
4,5,64,Male,Urban,501,3.66,15,2,0


#Part A — Operationalizing Ethics in Development

#Task 1 — Bias Assessment Before Deployment

    Train a baseline model to predict Flagged_For_Review.
    Measure positive prediction rates across Gender and City_Type.
    Identify if certain groups are unfairly targeted.


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [4]:
# Prepare data
X = pd.get_dummies(data.drop(columns=["Driver_ID","Flagged_For_Review"]), drop_first=True)
y = data["Flagged_For_Review"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train baseline model
model = RandomForestClassifier(n_estimators=50, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("Baseline Accuracy:", accuracy_score(y_test, y_pred))

Baseline Accuracy: 0.6066666666666667


In [5]:
# Bias check by Gender and Ethnicity
# Merge back with sensitive attributes for bias check
df_test = X_test.copy()
df_test["y_true"] = y_test
df_test["y_pred"] = y_pred
df_test["Gender"] = data.loc[y_test.index, "Gender"]
df_test["City_Type"] = data.loc[y_test.index, "City_Type"]

# Bias by Gender
print("\nPositive prediction rate by Gender:")
print(df_test.groupby("Gender")["y_pred"].mean())

# Bias by City_Type
print("\nPositive prediction rate by City_Type:")
print(df_test.groupby("City_Type")["y_pred"].mean())


Positive prediction rate by Gender:
Gender
Female    0.560284
Male      0.607639
Other     0.714286
Name: y_pred, dtype: float64

Positive prediction rate by City_Type:
City_Type
Metro    0.292553
Town     0.835616
Urban    0.809524
Name: y_pred, dtype: float64


#Task 2 — Ethics‑Aware Fairness Gate

    Create a fairness testing function that:
        Calculates demographic disparity.
        Fails the build if disparity > 0.08.


In [6]:
def fairness_check(df, sensitive_attr, threshold=0.08):
    rates = df.groupby(sensitive_attr)["y_pred"].mean()
    disparity = rates.max() - rates.min()
    print(f"{sensitive_attr} Disparity: {round(disparity, 3)}")
    return disparity <= threshold

print("\nFairness Checks:")
gender_fair = fairness_check(df_test, "Gender")
ethnic_fair = fairness_check(df_test, "City_Type")

if not (gender_fair and ethnic_fair):
    print("❌ Fairness check FAILED — Mitigation required.")
else:
    print("✅ Fairness check PASSED.")


Fairness Checks:
Gender Disparity: 0.154
City_Type Disparity: 0.543
❌ Fairness check FAILED — Mitigation required.


#Task 3 — Feature Importance Ethics Audit

    Output top feature importances.
    Flag socio‑demographic features (like Gender, City_Type) if they rank highly.
    Suggest mitigations such as masking, regularization, or balanced training data.


In [7]:
importances = pd.Series(model.feature_importances_, index=X.columns)
importances.sort_values(ascending=False).head(10)

,0
Trips_Completed,0.229226
Avg_Rating,0.225783
Age,0.187371
Complaints,0.161116
Accidents,0.089321
City_Type_Urban,0.041332
Gender_Male,0.030211
City_Type_Town,0.025864
Gender_Other,0.009776


#Task 4 — Ethical Development Documentation

    Draft two ethics user stories for the development team that explicitly incorporate bias mitigation and transparency.


#Part B — Operationalizing Ethics in Deployment

#Task 5 — Accountability via Prediction Logging

    Build a prediction API function that logs:
        Timestamp
        Model version
        Inputs
        Prediction outcome


In [8]:
import datetime
prediction_log = []
MODEL_VERSION = "1.0.0"

def predict_and_log(features):
    pred = model.predict(features)[0]
    log_entry = {
        "timestamp": datetime.datetime.now().isoformat(),
        "model_version": MODEL_VERSION,
        "input": features.to_dict(orient='records')[0],
        "prediction": int(pred)
    }
    prediction_log.append(log_entry)
    return pred

# Simulate one prediction
sample_applicant = X_test.iloc[[0]]
predict_and_log(sample_applicant)

# Inspect logs
prediction_log[:3]

[{'timestamp': '2025-08-13T12:21:51.087465',
  'model_version': '1.0.0',
  'input': {'Age': 56,
   'Trips_Completed': 1167,
   'Avg_Rating': 3.95,
   'Complaints': 9,
   'Accidents': 3,
   'Gender_Male': True,
   'Gender_Other': False,
   'City_Type_Town': False,
   'City_Type_Urban': True},
  'prediction': 1}]

#Task 6 — Post‑Deployment Monitoring

    Simulate continuous fairness metric tracking by group.
    Trigger an alert if disparity > 0.08.


#Task 7 — Privacy Protection

    Apply hashing to Driver_ID before storing logs to protect identity.


#Task 8 — Inclusivity Feedback Loop

    Simulate feedback submission from underrepresented driver groups.
    Show how these examples are added to the dataset for retraining.


Task 9 — Incident Response Simulation

    Create a mock ethical incident scenario (e.g., high flag rates for one city).
    Write a 4‑step incident plan:

  1. Detection
  2. Investigation
  3. Communication
  4. Remediation


Bonus Ideas

    Use SHAP values to explain why specific drivers are being flagged.
    Implement real‑time dashboard simulation for bias monitoring in deployment.
